In [39]:
from transformers import AutoProcessor, AutoModel
from torch import cat, save

from decord import VideoReader, cpu
from IPython.display import HTML

from utils import compute_clip_sim, get_samples

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
model_name = "microsoft/xclip-base-patch16-zero-shot"
processor, model = AutoProcessor.from_pretrained(model_name), AutoModel.from_pretrained(model_name)

In [5]:
filename = input()
vr = VideoReader(filename, ctx=cpu(0))
HTML(f'<video controls> <source src="{filename}" type="video/mp4"> </video>')

 videos/aerial.mp4


In [9]:
sample_interval = input() or 30   # in seconds
FPS = 24
print('total frames:', len(vr))

total frames: 6570


In [33]:
video_features = []
for i in range(0, len(vr), sample_interval*FPS):
    video_features.append(
        model.get_video_features(**processor(videos=list(get_samples(vr, start=i)), return_tensors="pt"))
    )

[0, 15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, 180, 195, 210, 225, 240, 255, 270, 285, 300, 315, 330, 345, 360, 375, 390, 405, 420, 435, 450, 465]
[720, 735, 750, 765, 780, 795, 810, 825, 840, 855, 870, 885, 900, 915, 930, 945, 960, 975, 990, 1005, 1020, 1035, 1050, 1065, 1080, 1095, 1110, 1125, 1140, 1155, 1170, 1185]
[1440, 1455, 1470, 1485, 1500, 1515, 1530, 1545, 1560, 1575, 1590, 1605, 1620, 1635, 1650, 1665, 1680, 1695, 1710, 1725, 1740, 1755, 1770, 1785, 1800, 1815, 1830, 1845, 1860, 1875, 1890, 1905]
[2160, 2175, 2190, 2205, 2220, 2235, 2250, 2265, 2280, 2295, 2310, 2325, 2340, 2355, 2370, 2385, 2400, 2415, 2430, 2445, 2460, 2475, 2490, 2505, 2520, 2535, 2550, 2565, 2580, 2595, 2610, 2625]
[2880, 2895, 2910, 2925, 2940, 2955, 2970, 2985, 3000, 3015, 3030, 3045, 3060, 3075, 3090, 3105, 3120, 3135, 3150, 3165, 3180, 3195, 3210, 3225, 3240, 3255, 3270, 3285, 3300, 3315, 3330, 3345]
[3600, 3615, 3630, 3645, 3660, 3675, 3690, 3705, 3720, 3735, 3750, 3765, 3780, 3795, 3810, 382

In [34]:
video_features = cat(video_features); video_features.shape

torch.Size([10, 512])

In [38]:
save(video_features, 'feature_vectors/aerial_10samples.pt')

In [24]:
text_features = model.get_text_features(**processor(text=["moving cars"], return_tensors="pt", padding=True))
logits = compute_clip_sim(video_features, text_features)
*logits, print('Content found at frame #:', logits.argmax())

Content found at frame #: tensor(9)


(tensor([0.1908, 0.1920, 0.2010, 0.1810, 0.1963, 0.1886, 0.1495, 0.1747, 0.1753,
         0.2351]),
 None)

In [35]:
text_features = model.get_text_features(**processor(text=["docked ship"], return_tensors="pt", padding=True))
logits = compute_clip_sim(video_features, text_features)
*logits, print('Content found at frame #:', logits.argmax())

Content found at frame #: tensor(9)


(tensor([0.1882, 0.1774, 0.1819, 0.1698, 0.1738, 0.1847, 0.1650, 0.1742, 0.1969,
         0.2153]),
 None)

In [31]:
6480/FPS/60

4.5